<a href="https://colab.research.google.com/github/axel-sirota/ml_ad_ai_course/blob/main/Hackathon/Hackathon_distillBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon: Finetuning DistillBERT

In this notebook we will fine-tune DistillBERT, a transformer based on BERT Googles model to create a toxicity model

You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

You can use any architecture you want! Good luck!

In [ ]:
!nvidia-smi

In [ ]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils

In [ ]:
import multiprocessing
import os
import random
import warnings

import keras.backend as K
import nltk
import numpy as np
import tensorflow as tf
from textblob import TextBlob

TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=100
buffer_size = 256
max_len = 50
# Batch size
batch_size = 256
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

In [ ]:
%%writefile get_data.sh
if [ ! -f toxic_comments.csv ]; then
  wget -O toxic_comments.csv https://www.dropbox.com/s/qecfi95tirln8sh/toxic_comments.csv?dl=0
fi


In [ ]:
!bash get_data.sh